<a href="https://colab.research.google.com/github/dequiroz/1MTR53_RobIA/blob/main/1_Manipulador_Cinem%C3%A1tica_Directa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=1FsTa2YzWwxY-HBkd2EOXAlkPJ1zpikWv" alt="drawing" width="150"/>


<p align="center">
<img src="https://drive.google.com/uc?export=view&id=1VBn7nKlruxCPLHH_UuD8B2pkvKP7JLRd" alt="drawing" width="800"/>

</br>

<img src="https://drive.google.com/uc?export=view&id=1uF_4x1YqLuS3M0mlh3NrMrEUHvjBMBFw" alt="drawing" width="800"/>
</p>

#Cinematica Directa para Manipulador de 2 grados de libertad (RR) - Análisis Geométrico
* Diego Quiroz Velasquez
* Sección Ingeniería Mecatrónica
* Pontificia Universidad Católica del Perú

---
Este archivo describe el método geométrico de cinemática directa y su aplicación para el análisis de un manipulador de 2 grados de libertad rotacionales. Se realiza un análisis simbólico para visualizar las ecuaciones matemáticas resultandos seguido por una aplicación numérica. Finalmente se realiza una simulación interactiva para comprender como la cinemática directa ayuda a determinar la posición del efector final del manipulador.



<img src="https://drive.google.com/uc?export=view&id=1vgFoFHEKMN0WcAFlKS2hPMg2KuBm3sp9" alt="drawing"/>

#Manipulador de 2 grados rotacionales RR

## 1. Introducción



Recordar el manipulador descrito

<p align="center">
<img src="https://drive.google.com/uc?export=view&id=1Gt7FztKupxVbmHPmauYFbJkDMeNEU9VO" alt="drawing" width="300"/></p>

La coordenadas del efector final se pueden determinar empleando geometría y están descrita por las ecuaciones

$\left [  \begin{matrix}
 x_2 \\ y_2
\end{matrix} \right ]=
\left [ \begin{matrix}
l_1 cos(\theta_1)+l_2 cos(\theta_1+\theta_2) \\ l_1 sin(\theta_1) + l_2 sin(\theta_1 + \theta_2)
\end{matrix}
\right ]$

Observar que estas ecuaciones describen un proceso de traslación en los ejes $X$ y $Y$ respectivamente. De forma adicional, el sistema de referencia del efector final presenta una rotación en el eje $Z$ con valor de $\theta_1 + \theta_2$

El proceso completo de cinemática directa se puede realizar empleando ecuaciones de transformación homogenea como se muestra a continuación

##2. Análisis simbólico

Para esté análisis se emplea la librería [sympy](https://www.sympy.org/en/index.html), la cual nos permitirá determinar las matrices de transformación homogeneas para cada eslabón así como su producto.

In [ ]:
import sympy as sp     # Se importa la biblioteca para el cálculo simbólico

In [ ]:
# =============================================================================
# Se definen funciones para representar transformaciones de traslación pura y
# rotación pura alrededor del eje Z
# =============================================================================
def transformacion_traslacion(x: sp.symbols, y: sp.symbols, z: sp.symbols) -> sp.Matrix:
    """
    Retorna una representación simbólica de la transformación homogénea
    de traslación pura

    Parametros (sp.symbols):
      x: translación en la dirección x
      y: translación en la dirección y
      z: translación en la dirección z
    """
    T = sp.Matrix([[1,0,0,x],
                   [0,1,0,y],
                   [0,0,1,z],
                   [0,0,0,1]])
    return T

def transformacion_rotacion_z(ang: sp.symbols) -> sp.Matrix:
    """
    Retorna una representación simbólica de la transformación homogénea
    de rotación pura alrededor de z

    Parametros (sp.symbols):
      ang: ángulo de rotación alrededor del eje z
    """
    T = sp.Matrix([[sp.cos(ang),-sp.sin(ang),0,0],
                   [sp.sin(ang), sp.cos(ang),0,0],
                   [0,0,1,0],
                   [0,0,0,1]])
    return T


In [ ]:
# Definición de variables representativas de la geometría del manipulador
q1, q2, l1, l2 = sp.symbols("q1 q2 l1 l2")

# Se definen las matrices de transformacion homogenea correspondientes empleando
# la notación:
#    Tab: Transformación del sistema 'b' con respecto al sistema 'a'

# Se definen la transformación del sistema 1 (codo) respecto al sistema 0 (base del robot)
T01 = transformacion_rotacion_z(q1)*transformacion_traslacion(l1,0,0)
# Se definen la transformación del sistema 2 (efector final) respecto al sistema 1 (codo)
T12 = transformacion_rotacion_z(q2)*transformacion_traslacion(l2,0,0)
# Se definen la transformación del sistema 2 (efector final) respecto al sistema 0 (base del robot)
T02 = sp.simplify(T01*T12)

# Mostrar las transformaciones homogéneas (display funciona con IPython)
print("T01:"); display(T01)
print("T12:"); display(T12)
print("T02:"); display(T02)

T01:


Matrix([
[cos(q1), -sin(q1), 0, l1*cos(q1)],
[sin(q1),  cos(q1), 0, l1*sin(q1)],
[      0,        0, 1,          0],
[      0,        0, 0,          1]])

T12:


Matrix([
[cos(q2), -sin(q2), 0, l2*cos(q2)],
[sin(q2),  cos(q2), 0, l2*sin(q2)],
[      0,        0, 1,          0],
[      0,        0, 0,          1]])

T02:


Matrix([
[cos(q1 + q2), -sin(q1 + q2), 0, l1*cos(q1) + l2*cos(q1 + q2)],
[sin(q1 + q2),  cos(q1 + q2), 0, l1*sin(q1) + l2*sin(q1 + q2)],
[           0,             0, 1,                            0],
[           0,             0, 0,                            1]])

Analizando la forma de la matriz $^0T_ 2$ se puede claramente observar:
* Traslación en los ejes $X$ y $Y$
* Rotación en el eje $Z$ con valor de $\theta_1 + \theta_2$

## 3. Cálculo numérico

El mismo procedimiento se puede realizar de forma numérica empleando numpy para el manejo de matrices y vectores en python

In [ ]:
import numpy as np

In [ ]:
# =============================================================================
# Se definen funciones para calcular las matrices de transformacio de traslación
# pura y rotación pura alrededor del eje Z
# =============================================================================
def transformacion_traslacion_num(x: float, y: float, z: float) -> np.ndarray:
  """
  Retorna una representación numérica de la transformación homogénea
  de traslación pura

  Parametros (float):
    x: translación en la dirección x
    y: translación en la dirección y
    z: translación en la dirección z
  """
  T = np.array([[1, 0, 0, x],
                [0, 1, 0, y],
                [0, 0, 1, z],
                [0, 0, 0, 1]])
  return T

def transformacion_rotacion_z_num(ang: float) -> np.ndarray:
  """
  Retorna una representación numérica de la transformación homogénea
  de rotación pura alrededor de z

  Parametros (float):
    ang: ángulo de rotación alrededor del eje z
  """
  T = np.array([[np.cos(ang), -np.sin(ang), 0, 0],
                [np.sin(ang), np.cos(ang), 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]])
  return T


In [ ]:
# =============================================================================
# Se define la función para calcular la cinemática directa del manipulador de
# 2 grados de libertad RR
# =============================================================================
def cinematica_directa_robot2D(q1: float, q2: float, l1: float, l2: float):
  """
  Calcula la cinemática directa del robot 2D

  Parámetros (float):
      q1: Ángulo de la articulación 1 en radianes.
      q2: Ángulo de la articulación 2 en radianes.
      l1: Longitud del eslabón 1.
      l2: Longitud del eslabón 2.

  Retorna:
      Una tupla de matriz 4x4 que representan las transformaciones homogénea de
      cada eslabón con respecto a la base del robot.
  """
  T01 = transformacion_rotacion_z_num(q1) @ transformacion_traslacion_num(l1, 0, 0)
  T12 = transformacion_rotacion_z_num(q2) @ transformacion_traslacion_num(l2, 0, 0)
  T02 = T01 @ T12
  return T01, T02


In [ ]:
from ipywidgets import interact

# =============================================================================
# Se emplea la función interact (funciona en Jupiter o Colab) para generar un
# sistema interactivo.
# =============================================================================
@interact(q1=(-180, 180, 1), q2=(-180, 180, 1), l1=(0, 10, 1), l2=(0, 10, 1))
def cinematica_directa_interactivo(q1, q2, l1, l2):
  # Aplicación de cinemática directa numérica
  q1 = np.deg2rad(q1)
  q2 = np.deg2rad(q2)
  T01, T02 = cinematica_directa_robot2D(q1, q2, l1, l2)

  # Mostrar el resultado
  print(np.round(T02,3))

interactive(children=(IntSlider(value=0, description='q1', max=180, min=-180), IntSlider(value=0, description=…

## 3. Simulación gráfica

En esta sección se emplea la librería [matplotlib](https://matplotlib.org/) para graficar un esquema del manipulador

In [ ]:
from ipywidgets import interact
import matplotlib.pyplot as plt

# =============================================================================
# Se emplea la función interact (funciona en Jupiter o Colab) para generar un
# sistema interactivo.
# =============================================================================
@interact(q1=(-180, 180, 1), q2=(-180, 180, 1), l1=(0, 10, 1), l2=(0, 10, 1))
def plot_robot(q1, q2, l1, l2):
  # Aplicación de cinemática directa numérica
  q1 = np.deg2rad(q1)
  q2 = np.deg2rad(q2)
  T01, T02 = cinematica_directa_robot2D(q1, q2, l1, l2)

  # Obtener las coordenadas del efector final
  x2 = T02[0, 3]
  y2 = T02[1, 3]

  # Obtener las coordenadas del codo
  x1 = T01[0, 3]
  y1 = T01[1, 3]

  # Graficar el robot
  plt.figure(figsize=(4, 4))
  plt.plot([0, x1, x2], [0, y1, y2], 'o-', color='purple', linewidth=2)
  plt.plot(0,0, x1, y1, marker='o', color='y', markersize=5)
  plt.xlabel('X')
  plt.ylabel('Y')
  plt.title('Robot de 2 grados de libertad RR')
  plt.xlim([-15, 15])
  plt.ylim([-15, 15])
  plt.grid(True)
  plt.show()

interactive(children=(IntSlider(value=0, description='q1', max=180, min=-180), IntSlider(value=0, description=…